In [4]:
import os
import numpy as np
import shutil
import json

from PIL import Image, ImageDraw, ImageFont
from pathlib import Path

In [2]:
def bbox_drawer(img_dir, label_dir, out_dir, names):
    img_files = os.listdir(img_dir)
    no_label_imgs = []
    no_label_img_cnt = 0
    for img_file in img_files:
        label_file = f'{img_file.split(".")[0]}.txt'
        img_file_path = img_dir / img_file
        label_file_path = label_dir / label_file
        pil_img = Image.open(img_file_path)
        pil_draw = ImageDraw.Draw(pil_img)
        width, height = pil_img.size

        try:
            labels = np.loadtxt(label_file_path)
        except:
            no_label_img_cnt += 1
            no_label_imgs.append(img_file)
            labels = np.array([])

        if labels.size > 0:
            if labels.ndim == 1:
                labels = np.expand_dims(labels, 0)
            x1 = (labels[:, 1] - labels[:, 3] / 2) * width
            y1 = (labels[:, 2] - labels[:, 4] / 2) * height
            x2 = (labels[:, 1] + labels[:, 3] / 2) * width
            y2 = (labels[:, 2] + labels[:, 4] / 2) * height

            for label in np.stack((labels[:, 0], x1, y1, x2, y2), axis=1).tolist():
                cat, xx1, yy1, xx2, yy2 = label
                pil_draw.rectangle((xx1, yy1, xx2, yy2), outline='red', width=3)
                pil_draw.text((xx1, yy1 - 22), f'{names[int(label[0])]}', fill='red', font=ImageFont.truetype('arial.ttf', size=20))
        
        img_out_file_path = out_dir / img_file
        pil_img.save(img_out_file_path)

    print(f'{no_label_img_cnt} image file dont have labels')
    return no_label_imgs, no_label_img_cnt

In [3]:
def initial_extract(img_dir, label_dir, out_dir):
    licenses = [
        {
            "name": "",
            "id": 0,
            "url": ""
        }
    ]

    info_ = [
        {
            "contributor": "",
            "date_created": "",
            "description": "",
            "url": "",
            "version": "",
            "year": ""
        }
    ]

    categories = [
        {
            "id": 1,
            "name": "Buoy",
            "supercategory": ""
        },
        {
            "id": 2,
            "name": "Sailing yacht",
            "supercategory": ""
        },
        {
            "id": 3,
            "name": "Boat",
            "supercategory": ""
        },
        {
            "id": 4,
            "name": "Structure",
            "supercategory": ""
        },
        {
            "id": 5,
            "name": "Other",
            "supercategory": ""
        },
        {
            "id": 6,
            "name": "Channel Marker",
            "supercategory": ""
        },
        {
            "id": 7,
            "name": "Speed Warning Sign",
            "supercategory": ""
        },
        {
            "id": 8,
            "name": "Extra boat",
            "supercategory": ""
        }
    ]

    img_idx = 0
    annot_idx = 0

    imgs_list = []
    annots_list = []

    for label_file in os.listdir(label_dir):
        label_file_ = os.path.join(label_dir, label_file)
        img_file_ = os.path.join(img_dir, f'{label_file.split(".")[0]}.jpg')
        img = Image.open(img_file_)
        image_w, image_h = img.size

        imgs_list.append({
            'id': img_idx,
            'width': image_w,
            'height': image_h,
            'file_name': f'{label_file.split(".")[0]}.jpg',
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        })

        with open(label_file_, 'r') as label_f:
            labels = label_f.readlines()

            for label in labels:
                cat, xc, yc, label_normalized_w, label_normalized_h = list(map(lambda x: int(x) if len(x) == 1 else float(x), label.split()))
                label_w, label_h = image_w * label_normalized_w, image_h * label_normalized_h
                xmin, ymin = (image_w * xc) - (label_w / 2), (image_h * yc) - (label_h / 2)
                
                xmin = 0 if xmin < 0 else xmin
                ymin = 0 if ymin < 0 else ymin

                annots_list.append({
                    'id': annot_idx,
                    'image_id': img_idx,
                    'category_id': cat + 1,
                    'area': int(label_h * label_w),
                    'bbox': [
                        xmin,
                        ymin,
                        label_w,
                        label_h
                    ],
                    'iscrowd': 0,
                    'attributes': {
                        'type': '',
                        'occluded': False
                    }
                })

                annot_idx += 1

        img_idx += 1

    out_dict = {
        'licenses': licenses,
        'info': info_,
        'categories': categories,
        'images': imgs_list,
        'annotations': annots_list
    }

    with open(os.path.join(out_dir, 'val.json'), 'w') as out_f:
        print(os.path.join(out_dir, 'val.json'))
        json.dump(out_dict, out_f)